In [1]:
%load_ext autoreload
%autoreload 2
from RNN2DFA.LSTM import LSTMNetwork
# from GRU import GRUNetwork
from RNN2DFA.RNNClassifier import RNNClassifier
from RNN2DFA.Training_Functions import mixed_curriculum_train
from RNN2DFA.Training_Functions import make_test_set,make_train_set_for_target
from lexr.RNNexplainer import Explainer
import pandas as pd
import lexr.LTL2DFA as ltlf2dfa
from RNN2DFA.Extraction import extract
import pickle
from samples2ltl.utils.Traces import Trace

In [2]:
# Reber grammar
from lexr.specific_examples import Reber_Grammar
rg=Reber_Grammar()
alphabet=rg.alphabet
generator_dfa=rg
sample_train_set=[]
for i in range(100):
    seq, _, _ = rg.get_one_example(maxLength=10)
    sample_train_set.append(seq)
    # print(rg.classify_word(rg.sequenceToWord(seq)))

In [3]:
import lexr.specific_examples
generator_dfa=lexr.specific_examples.Text_Classification()
target_formula = generator_dfa.target_formula
alphabet = generator_dfa.alphabet
query_formulas = generator_dfa.query_formulas
print(target_formula)

['|(x192,F(x101))', '|(x341,|(x49,x797))', '|(x40,F(x19))', '|(x591,F(x5))', '|(x528,F(x169))', '|(x100,F(x4))', '|(x172,F(x2))', '|(x482,F(x124))', '|(x47,x736)', '|(x47,x720)', '|(x124,F(x19))', 'F(|(x15,x21))', 'F(|(x323,x4))', '|(x797,F(x26))', 'F(x912)', 'U(!(x3),x2)', '|(x113,F(x19))', '|(x192,F(x19))', 'F(|(x148,x71))', '|(x188,F(x21))', '|(x235,F(x48))', '|(x16,F(x4))', 'F(x26)', '|(x298,|(x210,x3))', 'F(x4)', '|(x3,F(x21))', '|(x70,F(x12))', '|(x100,F(x3))', '|(x324,|(x2,x420))', 'F(x5)', '|(x235,F(x650))', 'F(|(x106,x21))', '|(x13,F(x96))', '|(x488,F(x9))', 'F(x21)', 'F(|(x122,x31))', '|(x889,F(x35))', '|(x22,F(x12))', '|(x28,x438)', 'F(|(x2,x958))', '|(x344,x47)', 'F(x197)', '|(x192,x583)', '|(x73,F(x7))', '|(x641,F(x2))', '|(x178,F(x2))', 'F(x96)', '|(x637,F(x47))', 'F(x388)', '|(x773,F(x26))', 'F(x38)', '|(x83,F(x873))', 'F(|(x147,x47))', '|(x192,x483)', '|(x104,F(x337))', '|(x13,F(x9))', '|(x113,F(x16))', '|(x113,F(x3))', '|(x70,F(x96))', '|(x175,F(x19))', '|(x106,F(x22))

In [4]:
from sklearn.model_selection import train_test_split
def dict2lists(dictionary):
    X,y=[],[]
    for key in dictionary:
        X.append(key)
        y.append(dictionary[key])
    return X,y

def lists2dict(x,y):
    # both x and y should have same length
    assert len(x)==len(y), "Error dimension"
    d={}
    n=len(x)
    for idx in range(n):
        d[x[idx]]=y[idx]
    return d

maximum_sequence_length = 50
num_layers = 2
num_hidden_dim = 10
input_dim = 3
iterations = 1
stop_threshold = 0.0005
RNNClass = LSTMNetwork
if(target_formula == "DNA sequence"):
    num_layers = 5
    num_hidden_dim = 10
    input_dim = 5
    stop_threshold = 0.1
    RNNClass = LSTMNetwork

if(target_formula == "Text classification" or target_formula == "Deceptive opinion"):
    num_layers = 5
    num_hidden_dim = 20
    input_dim = 10
    stop_threshold = 0.1
    RNNClass = LSTMNetwork


# for each example, specify a different generating function
file_name = "benchmarks/" + target_formula.replace(" ", "_")+".pkl"

if not os.path.isfile(file_name):

    if(target_formula == "balanced parentheses"):

        train_set = generator_dfa.get_balanced_parantheses_train_set(8000, 2, 50, max_train_samples_per_length=3000,
                                                                     search_size_per_length=2000, lengths=[i for i in range(maximum_sequence_length+1)])

    elif(target_formula == "email match"):

        train_set = make_train_set_for_target(generator_dfa.classify_word, alphabet, lengths=[i for i in range(maximum_sequence_length+1)],
                                              max_train_samples_per_length=1000,
                                              search_size_per_length=3000, deviation=200)

        # generate more examples that match the regular expression
        matching_strings = generator_dfa.generate_matching_strings(
            n=10800, max_length=50)
        for string in matching_strings:
            train_set[string] = True

    elif(target_formula == "alternating bit protocol"):

        train_set = make_train_set_for_target(generator_dfa.classify_word, alphabet, lengths=[i for i in range(maximum_sequence_length+1)],
                                              max_train_samples_per_length=1000,
                                              search_size_per_length=3000, deviation=250)

        # generate more examples that match the regular expression
        matching_strings = generator_dfa.generate_matching_strings(
            n=105000, max_sequence_length=50)
        for string in matching_strings:
            train_set[string] = True
    elif(target_formula == 'G(a->X(b))'):
        train_set = make_train_set_for_target(generator_dfa.classify_word, alphabet, lengths=[i for i in range(maximum_sequence_length+1)],
                                              max_train_samples_per_length=1000,
                                              search_size_per_length=3000, deviation=20)
    elif(target_formula == "DNA sequence" or target_formula == "Text classification" or target_formula == "Deceptive opinion"):
        train_set = generator_dfa.dict
        
    else:
        train_set = make_train_set_for_target(generator_dfa.classify_word, alphabet, lengths=[i for i in range(maximum_sequence_length+1)],
                                              max_train_samples_per_length=100,
                                              search_size_per_length=300, deviation=20)

    # now save the dataset to file
    with open(file_name, "wb") as f:
        pickle.dump(train_set, f, protocol=3)

else:
    # load the dataset
    print("loading from previously stored benchmarks")

    def load_obj(name):
        with open(name, "rb") as f:
            return pickle.load(f)
    train_set = load_obj(file_name)


# print(train_set)
# print(generator_dfa.alphabet)



loading from previously stored benchmarks


In [5]:
print(target_formula)




# print ratio
cnt = 0
examples_per_length = [0 for i in range(51)]
for key in train_set:
    if(train_set[key]):
        cnt += 1
    try:
        if("x" in key):
            examples_per_length[len(key.split('x')[1:])] += 1
        else:
            examples_per_length[len(key)] += 1
    except:
        pass

total_samples = len(train_set)
print("out of ", total_samples, " sequences", cnt,
      " are positive. (percent: ", float(cnt/total_samples), ")")
print("examples per length:", examples_per_length)

# split train:test
X, y = dict2lists(train_set)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
train_set = lists2dict(X_train, y_train)
test_set = lists2dict(X_test, y_test)
train_set_size = len(train_set)
test_set_size = len(test_set)

# intentionally pushing "" (empty string) in train_set
if('' not in train_set):
    try:
        train_set[''] = test_set['']
        print("Empty string status:", train_set[''])
    except:
        print("Empty string is not in test test also")
else:
    print("Empty string was already included in train set")
    print("Empty string status:", train_set[''])

print("size of train set:", train_set_size)
print("size of test set:", test_set_size)

print("configurations: layers: ", num_layers,
          "hidden dimension: ", num_hidden_dim,
          "input dim: ", input_dim,
          "network: ", RNNClass,
          "stop threshold: ", stop_threshold)


# define rnn
rnn = RNNClassifier(alphabet, num_layers=num_layers,
                    hidden_dim=num_hidden_dim, RNNClass=RNNClass, input_dim=input_dim, target=target_formula)

# train the model
if not os.path.isfile("model/"+target_formula+".model"):
    mixed_curriculum_train(rnn, train_set, stop_threshold=stop_threshold)
    rnn.save_model()
else:
    print("loading already saved model")
    rnn.load_model()
    
rnn.renew()
dfa_from_rnn = rnn
# statistics

def percent(num, digits=2):
    tens = pow(10, digits)
    return int(100*num*tens)/tens

print("testing on train set, i.e. test set is train set")
# we're printing stats on the train set for now, but you can define other test sets by using
# make_train_set_for_target

pos = 0
rnn_target = 0
for w in test_set:
    if generator_dfa.classify_word(w):
        pos += 1

    if dfa_from_rnn.classify_word(w) == generator_dfa.classify_word(w):
        rnn_target += 1
test_acc = percent(rnn_target/test_set_size)
print("rnn score against target on test set:                             ",
        rnn_target, "("+str(test_acc)+")")
        



Text classification
out of  5043  sequences 574  are positive. (percent:  0.11382113821138211 )
examples per length: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Empty string is not in test test also
size of train set: 4034
size of test set: 1009
configurations: layers:  5 hidden dimension:  20 input dim:  10 network:  <class 'RNN2DFA.LSTM.LSTMNetwork'> stop threshold:  0.1
The dy.parameter(...) call is now DEPRECATED.
        There is no longer need to explicitly add parameters to the computation graph.
        Any used parameter will be added automatically.
loading already saved model
testing on train set, i.e. test set is train set
rnn score against target on test set:                              986 (97.72)


In [7]:
from samples2ltl.utils.SimpleTree import Formula

timeout = 30
maximum_sequence_length = 500
maximum_formula_depth = 50
epsilon = 0.05
delta = 0.05


# use a query LTL formula
text_formula = 'F(|(x230,x3))'
query_formula = Formula.convertTextToFormula(text_formula)
print(query_formula)

# print(query_formula)
# query_dfa=ltlf2dfa.translate_ltl2dfa(alphabet=["X_" + str(character) for character in alphabet],formula=query_formula, token="bal")
# print(query_dfa)
"""  
Create initial samples
"""


from lexr.RNNexplainer import Traces
traces=Traces(rnn, alphabet, token="bal")
traces.label_from_network([])
traces.write_in_file()




from PACTeacher.pac_teacher import PACTeacher as Teacher 
explainer=Explainer(alphabet=[character for character in alphabet], token="bal")
teacher = Teacher(dfa_from_rnn,epsilon=.03, delta=.03, max_trace_length=20, max_formula_depth=10, query_dfa=query_formula)



import time
start_time=time.time()
from multiprocessing import Process, Queue
explainer, flag, learner_time, verification_time = teacher.teach(explainer, traces, timeout = timeout, verbose = False)
end_time=time.time()


print("\n\nepsilon=", teacher.epsilon, "delta=", teacher.delta, "max_trace_length=", teacher.max_trace_length)
print("query:", text_formula)
print("final ltl: ", explainer.ltl)




new_delta = None
new_epsilon = None
if(not flag):
    print("incomplete formula")
    new_delta, new_epsilon = teacher.calculate_revised_delta_and_epsilon()
    print(new_delta, new_epsilon)


print("returned counterexamples:", teacher.returned_counterexamples)

print("time learner:", learner_time)
print('time verifier:', verification_time)
print("Random words:", teacher.number_of_words_checked)
print("Time taken:", end_time-start_time)
print()



run_lstar = False
if(run_lstar):
        print("\n\n\n\n")
        # compare with dfa from lstar_algorithm
        dfa_from_rnn.renew()
        start_time_lstar = time.time()
        dfa_lstar, lstar_flag = extract(rnn, query=query_formula, max_trace_length=maximum_sequence_length, epsilon=delta,
                                        delta=delta, time_limit=timeout, initial_split_depth=10, starting_examples=[])
        end_time_lstar = time.time()

        dfa_lstar = dfa_lstar.minimize_()
        dfa_lstar.draw_nicely(
            filename=target_formula+":"+text_formula+"_"+str(epsilon)+"_"+str(delta))

        
        print("\nTime taken to extract lstar-dfa:",
                end_time_lstar-start_time_lstar)
        print("returned flag:", lstar_flag)
        print("transitions:->")
        # print(dfa_lstar.delta)
        num_lstar_states = len(dfa_lstar.Q)
        print("number of states of the dfa:", num_lstar_states)


performance_explanation_with_rnn = performance_rnn_with_groundtruth = performance_explanation_with_groundtruth = 0
lstar_performance_explanation_with_rnn = lstar_performance_explanation_with_groundtruth = 0

test_set_size = 0
for w in test_set:

    

    dfa_from_rnn.renew()

    test_set_size += 1
    verdict_rnn = dfa_from_rnn.classify_word(w)
    verdict_target = generator_dfa.classify_word(w)
    trace_vector = []
    if("x" in w):
        for letter in w.split("x")[1:]:
            if(target_formula == "Text classification" and letter == '0'):
                continue
            trace_vector.append([alphabet[i] == "x" + letter for i in range(len(alphabet))])
    else: 
        for letter in w:
            trace_vector.append([alphabet[i] == letter for i in range(len(alphabet))])
    if(len(w) == 0):
        trace = Trace([[False for _ in alphabet]])
    else:
        trace = Trace(trace_vector)

    verdict_ltl = trace.evaluateFormulaOnTrace(explainer.formula)
    verdict_query =  trace.evaluateFormulaOnTrace(query_formula)

    if(run_lstar):
        verdict_lstar = dfa_lstar.classify_word(w)

    if (verdict_rnn and verdict_query) == verdict_ltl:
        performance_explanation_with_rnn += 1
    if verdict_rnn == verdict_target:
        performance_rnn_with_groundtruth += 1
    if verdict_ltl == (verdict_target and verdict_query):
        performance_explanation_with_groundtruth += 1
    if(run_lstar):
        if (verdict_rnn and verdict_query) == verdict_lstar:
            lstar_performance_explanation_with_rnn += 1
        # else:
        #     print(w, verdict_lstar, verdict_rnn, verdict_query)
        if verdict_lstar == (verdict_target and verdict_query):
            lstar_performance_explanation_with_groundtruth += 1
        # else:
        #     print(w, verdict_lstar, verdict_target, verdict_query)


if(test_set_size != 0):
    print("Explanation matches RNN:", str(
        percent(performance_explanation_with_rnn/test_set_size)))

    print("RNN matches ground truth:", str(
        percent(performance_rnn_with_groundtruth/test_set_size)))

    print("Explanation matches ground truth:", str(
        percent(performance_explanation_with_groundtruth/test_set_size)))

    if(run_lstar):
        print("Lstar matches RNN:", str(
            percent(lstar_performance_explanation_with_rnn/test_set_size)))

        print("Lstar matches ground truth:", str(
            percent(lstar_performance_explanation_with_groundtruth/test_set_size)))



if(not run_lstar):
    num_lstar_states = None
    start_time_lstar = 0
    end_time_lstar = 0
    lstar_performance_explanation_with_rnn = 0
    lstar_performance_explanation_with_groundtruth = 0
    lstar_flag = False




F(|(x230,x3))
learned LTL formula:false
0  iteration complete



learned LTL formula:true
1  iteration complete



learned LTL formula:x774
2  iteration complete



learned LTL formula:(F x230)


epsilon= 0.03 delta= 0.03 max_trace_length= 20
query: F(|(x230,x3))
final ltl:  (F x230)
returned counterexamples: ['x774x15x230', '', 'x774']
time learner: 4.594047784805298
time verifier: 12.810323238372803
Random words: 541
Time taken: 28.076335906982422

Explanation matches RNN: 95.83
RNN matches ground truth: 97.72
Explanation matches ground truth: 96.03


In [11]:
f = open("benchmarks/query/" + target_formula + ".txt")
query_formulas = f.readline()[1:-1].split(", ")
query_formulas = list(set([query_formula[1:-1] for query_formula in query_formulas]))
f.close()
print(query_formulas)

['X(x0)', 'F(x0)', '!(x1)', '!(x2)', 'x0', 'x2']
